# 提示工程（Prompt engineering）一些技巧

提示工程是人工智慧中的一個概念，特別是自然語言處理 (NLP) 領域。 在提示工程中，透過用戶輸入任務的描述。引導與干涉模型產出正確的結果。這種以提示為基礎的學習（prompt-based learning）」是運用大型語言模型的主要方式。提示工程有多種技巧 (https://github.com/dair-ai/Prompt-Engineering-Guide) 以下示範幾種典型的技巧。

In [1]:
import openai
import os
from dotenv import load_dotenv

# 載入環境變數內容
load_dotenv()
# 呼叫 OpenAI ChatComplete API 
def call_openai_api(prompt, model_name=os.getenv('DEPLOYMENT_NAME'), temperature=0, max_token=100):
    openai.api_type = "azure"
    openai.api_version = "2023-05-15"
    
    openai.api_key = os.getenv('OPENAI_API_KEY')
    openai.api_base =  os.getenv('OPENAI_API_BASE')
                                    
    response  = openai.ChatCompletion.create( 
        engine = model_name ,
        messages =  [{'role': 'user', 'content': prompt}],
        temperature = temperature,
        max_tokens=max_token)

    return (response['choices'][0]['message']['content'])





# 思維鍊 (Chain of Thought, CoT)

大型語言模型在面對一個需要數學運算與推理的問題任務，直接生成的答案往往是錯誤的，但在過去幾年研究發現，將決問題的步驟逐一拆解，搭配精心設計的提示，能夠大幅提高大型語言模型生成答案的正確性。這種引導大型語言模型思路的技巧稱為思維鍊 (Chain of Thought, COT)，以下是以 GPT-3.5 Turbo 透過 ChatComplete API 的思維鍊範例。

In [2]:
# 這個提示得到的答案是錯的

PROMPT_ZERO_SHOT =  "Q:一個盒子內放了16顆球，一半是高爾夫球。" \
                    "一半的高爾夫球是藍色的，請問多有少個藍色高爾夫球在盒子裏?\n" \
                    "A:"

response = call_openai_api(PROMPT_ZERO_SHOT, temperature=0, max_token=200)

print(response)

盒子裡有8顆藍色高爾夫球。


In [3]:
# 這個提示得到的答案也是錯的

PROMPT_ZERO_SHOT =  "Q: 老李有 16 個孩子，其中四分之一是女兒，都沒結婚，而兒子中一半已婚，已婚的兒子都各有兩個兒子，請問老李兒子有幾位孫子?\n" \
                    "A:"

response = call_openai_api(PROMPT_ZERO_SHOT, temperature=0, max_token=200)

print(response)

老李有 12 個兒子和 4 個女兒，其中 3 個兒子已婚，共有 6 個孫子。


以下開始使用思維鍊 (CoT) 引導模型推理

In [4]:
# 以 Few Shots 搭配思維鍊 (CoT) , 引導出正確的答案

PROMPT_FEW_SHOTS =  "Q: 一個盒子內放了16顆球，其中一半的球是高爾夫球。" \
                    "一半的高爾夫球是藍色的，請問多有少個藍色高爾夫球在盒子裏?\n" \
                    "A: 盒子內一半是高爾夫球，因此高爾夫球的數量是 16 / 2 = 8顆，由於一半高爾夫球是藍色的，所以 8 / 2 = 4 顆，答案是 4 顆藍色高爾夫球\n" \
                    "Q: 老李有 16 個孩子，其中四分之一是女兒，都沒結婚，而兒子中一半已婚，已婚的兒子都各有兩個兒子，請問老李兒子有幾位孫子?\n" \
                    "A:"

response = call_openai_api(PROMPT_FEW_SHOTS, temperature=0, max_token=200)

print(response)

老李有 16 x 1/4 = 4 個女兒，因此有 16 - 4 = 12 個兒子。
已婚的兒子有 12 x 1/2 = 6 位。
每位已婚兒子都有 2 個兒子，因此總共有 6 x 2 = 12 位孫子。
答案是 12 位孫子。


# 更多思維鍊 (CoT) 提示的案例




# Program-aided Language Models
PAL (Program-aided Language) Models 是思維鍊的技巧之一，是利用程式語言的數學公式來引導大型語言模型推理 (https://cobusgreyling.medium.com/pal-program-aided-large-language-models-30db3e59f796)， 原本 GPT 是大型語言模型原本並非設計出來處理數學問題，隨著模型版本更新，數學推理能力也逐漸增強，但本範例仍僅是透過 PAL Models 這種思維鍊的技巧，展示大型語言模型推理的能力。

In [5]:
# 這個提示雖然得到正確的方程式，但得到的答案確是錯誤的

PROMPT_ZERO_SHOT = "Q: 數隻雞與數隻兔關在一個龍子裡面，只知道一共5隻動物，一共有 18 隻腳? 請問龍子裏有幾隻兔? 幾隻雞? \n" \
                   "A:"

response = call_openai_api(PROMPT_ZERO_SHOT, temperature=0, max_token=600)

print(response)

假設有 x 隻雞和 y 隻兔，則有以下方程式：
x + y = 5 (總動物數)
2x + 4y = 18 (總腳數)

解方程式得到：
x = 2, y = 3

因此，龍子裡有 3 隻兔和 2 隻雞。


In [6]:
# 透過程式語言語法表達數學式，以 PAL Models 思維鍊方式引導模型找出答案

PROMPT_FEW_SHOTS = "Q: 數隻雞與數隻兔關在一個籠子裡面，只知道一共5隻動物，一共有 18 隻腳? 請問籠子裏有幾隻雞? \n" \
                   "A:Rabbit + Chicken = 5 \n" \
                    "Rabbit = 5 - Chicken \n" \
                    "Rabbit * 4 + Chicken * 2 = 18 \n" \
                    "(5 - Chicken) * 4 + Chicken * 2 = 18 \n" \
                    "20 - Chicken * 4 + Chicken * 2 = 18 \n" \
                    "20 - 18 = Chicken * 4 - Chicken * 2 \n" \
                    "2 = (4 -2) * Chicken  \n" \
                    "2 = 2 * Chicken  \n" \
                    "Chicken = 1 \n" \
                    "Rabbit = 5 - 1 \n" \
                    "Rabbit = 4 \n" \
                    "答案是 1 隻雞，4 隻兔子 \n" \
                    "###\n" \
                    "Q: 不明數量的蟑螂與數隻烏龜關在一個籠子裡面，已隻共6隻動物，並且共有 32 隻腳? 請問籠子裏有幾隻蟑螂? \n" \
                    "A:"

response = call_openai_api(PROMPT_FEW_SHOTS, temperature=0, max_token= 600)

print(response)

Cockroach + Turtle = 6 
Cockroach * 6 + Turtle * 4 = 32 
(6 - Turtle) * 6 + Turtle * 4 = 32 
36 - Turtle * 6 + Turtle * 4 = 32 
36 - 32 = Turtle * 2 
4 = 2 * Turtle 
Turtle = 2 
Cockroach = 6 - 2 
Cockroach = 4 
答案是 4 隻蟑螂，2 隻烏龜。


In [7]:
# 類似雞兔同籠題型也可以正確處理

PROMPT_FEW_SHOTS = "Q: 數隻雞與數隻兔關在一個籠子裡面，只知道一共5隻動物，一共有 18 隻腳? 請問籠子裏有幾隻雞? \n" \
                   "A:Rabbit + Chicken = 5 \n" \
                    "Rabbit = 5 - Chicken \n" \
                    "Rabbit * 4 + Chicken * 2 = 18 \n" \
                    "(5 - Chicken) * 4 + Chicken * 2 = 18 \n" \
                    "20 - Chicken * 4 + Chicken * 2 = 18 \n" \
                    "20 - 18 = Chicken * 4 - Chicken * 2 \n" \
                    "2 = (4 -2) * Chicken  \n" \
                    "2 = 2 * Chicken  \n" \
                    "Chicken = 1 \n" \
                    "Rabbit = 5 - 1 \n" \
                    "Rabbit = 4 \n" \
                    "答案是 1 隻雞，4 隻兔子 \n" \
                    "###\n" \
                    "Q: 不知道有多少隻蜘蛛與不明數量的蛇關在一同個籠子裡面，已隻共6隻動物，並且共有 24 隻腳? 請問籠子裏有幾隻蛇幾隻蜘蛛? \n" \
                    "A:"

response = call_openai_api(PROMPT_FEW_SHOTS, temperature=0, max_token= 600)

print(response)

Spider + Snake = 6 
Spider = 6 - Snake 
Spider * 8 + Snake * 0 = 24 
(6 - Snake) * 8 + Snake * 0 = 24 
48 - Snake * 8 + Snake * 0 = 24 
48 - 24 = Snake * 8 - Snake * 0 
24 = 8 * Snake  
Snake = 3 
Spider = 6 - 3 
Spider = 3 
答案是 3 隻蛇，3 隻蜘蛛。


# 常識推論 (Commonsense Reasoning)

2019 年使用 OpenAI GPT-2 語言模型架構並以現有人類常識知識庫加以訓練 https://www.quantamagazine.org/common-sense-comes-to-computers-20200430/ ，即可達到接近人類水準的常識性推理。然而大型語言模型仍需要補充更多高品質的知識，才能達到人類水準的常識推論能力。

In [8]:
# 得到錯誤的答案
PROMPT = "對於一個在 1968 年進行自然考試的學生而言，在考卷中有一題 '太陽系有幾大行星？' 這題正確答案是什麼? "
response = call_openai_api(PROMPT,temperature=0 , max_token=800)

print(response)

太陽系有八大行星。


In [9]:
# 補充知識後的到正確的答案
PROMPT = "2006年8月國際天文聯合會將冥王星地位降級矮行星\n" \
         "###\n" \
         "對於一個在 1968 年進行自然考試的學生而言，在考卷中有一題 '太陽系有幾大行星？' 這題正確答案是什麼? "
response = call_openai_api(PROMPT,temperature=0 , max_token=800)

print(response)

答案是九大行星。在 2006 年之前，太陽系被認為有九大行星，包括水星、金星、地球、火星、木星、土星、天王星、海王星和冥王星。然而，國際天文聯合會在 2006 年將冥王星的地位降級為矮行星，因此現在太陽系只有八大行星。


In [10]:
# 基本知識不足，仍會得到錯誤的答案
PROMPT = "2006年8月國際天文聯合會將冥王星地位降級矮行星\n" \
         "###\n" \
         "對於一個在 1890 年進行自然考試的學生而言，在考卷中有一題 '太陽系有幾大行星？' 這題正確答案是什麼? "
response = call_openai_api(PROMPT,temperature=0 , max_token=800)

print(response)

答案是九大行星。但是自2006年8月以后，冥王星的地位被降级为矮行星，因此现在太阳系中只有八大行星。


In [11]:
# 補充更多的知識，以得到正確的常識推論結果
PROMPT = "1846年9月德國天文學家發現海王星，太陽系成為八大行星\n" \
         "1930年2月美國科學家發現冥王星，太陽系成為九大行星\n" \
         "2006年8月國際天文聯合會將冥王星地位降級矮行星，太陽系成為八大行星\n" \
         "###\n" \
         "對於一個在 1890 年進行自然考試的學生而言，在考卷中有一題 '太陽系有幾大行星？' 這題正確答案是什麼? "
response = call_openai_api(PROMPT,temperature=0 , max_token=800)

print(response)

當時的正確答案是八大行星。


In [13]:
# 得到錯誤的答案
PROMPT = "游泳池加熱至攝氏 100 度，水獺與人類游泳選手在這個游泳池比賽 100 公尺游泳，誰會勝出? "
response = call_openai_api(PROMPT,temperature=0 , max_token=800)

print(response)

這個問題有點棘手，因為水獺和人類游泳選手的游泳速度取決於很多因素，例如體型、技巧、力量和耐力等等。而且，游泳池加熱至攝氏 100 度可能對人類游泳選手產生不良影響，因為這樣高的溫度可能會導致熱衰竭或中暑等問題。

總的來說，如果假設水獺和人類游泳選手都能夠在這樣高的溫度下游泳，那麼誰會勝出取決於他們的個人能力和表現。一般來說，水獺在水中游泳的速度比人類游泳選手快，因為它們的身體設計和游泳技巧使它們更加適應水中環境。但是，如果比賽是在游泳池中進行，人類游泳選手可能會利用自己的技巧和策略來贏得比賽。

總之，這個問題沒有一個確定的答案，因為太多的因素會影響比賽的結果。


In [2]:
# 補充更多的知識，以得到正確的常識推論結果
PROMPT = "攝氏 100 度下水溫下多數地球上的動物無法存活太久" \
         "###\n" \
         "游泳池加熱至攝氏 100 度，水獺與人類游泳選手在這個游泳池比賽 100 公尺游泳，誰會勝出? "
response = call_openai_api(PROMPT,temperature=0 , max_token=800)

print(response)

在攝氏 100 度下，多數地球上的動物無法存活太久，包括水獺和人類游泳選手。因此，這個比賽無法進行，也無法有勝負結果。
